In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import promonet


In [ ]:
audio = promonet.load.audio('/files10/max/promonet/data/cache/daps/0010/000026-100.wav')
pitch, periodicity, loudness, ppg = promonet.preprocess.from_audio(audio, gpu=0)


In [ ]:
promonet.plot.from_features(audio, pitch, periodicity, loudness, ppg)


In [ ]:
pitch


In [ ]:
import ppgs
import matplotlib.pyplot as plt
import matplotlib
import torch
plt.imshow(ppgs.from_file('/files10/max/promonet/data/cache/daps/0010/000026-100.wav', gpu=0).cpu())



In [ ]:
plt.imshow(torch.load('/files10/max/promonet/data/cache/daps/0010/000026-100-ppg.pt'))
